In [ ]:
!pip install OpenNMT-py

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

In [ ]:
%cd /content/drive/MyDrive/TP_NMT/tp_nmt/


## Préparation des données

**Q1**
> Ces fichiers contiennent les données que nous allons utilisées pour entraîner notre réseau de neurones.
Sur chaque ligne, nous avons la phrase source et la phrase target.

In [ ]:
%%shell
cd BTEC-en-fr
for i in ./train/* ./dev/* ./test/*
do
  awk -F '\' '{print $NF}' $i > ${i}_out.clean.txt 
done

In [ ]:
%%shell
perl ./tokenizer.perl -l en -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.en.txt_out.clean.txt > BTEC-en-fr/train/IWSLT10_BTEC.train.en.txt_out.tok.txt
perl ./tokenizer.perl -l fr -lc < ./BTEC-en-fr/train/IWSLT10_BTEC.train.fr.txt_out.clean.txt > BTEC-en-fr/train/IWSLT10_BTEC.train.fr.txt_out.tok.txt

perl ./tokenizer.perl -l en -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.clean.txt > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt
perl ./tokenizer.perl -l fr -lc < ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.clean.txt > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt

perl ./tokenizer.perl -l en -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt_out.clean.txt > BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt_out.tok.txt
perl ./tokenizer.perl -l fr -lc < ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.clean.txt > BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.tok.txt

**Q2**
> Dans les fichiers .tok, les symboles de ponctuations sont étiquetés et certains espaces sont ajoutés autour d'eux dans le but d'avoir une data plus structurée et donc de facilité son traitement.

<span style="color:red"> Ca veut dire quoi data plus structurés? Le but est de pouvoir construire un vocabulaire de taille raisonnable à partir de ces donnée. Si on ne sépare pas les mots et la ponctuation on sera obligé à avoir plu de tokens dans le vocabularie. E.g. "Salut", "Salut!", "Salut,", "Salut?", etc. </span>

**Q3**

> Le training set permet d'entraîner le réseaux via la modification des poids

> Le dev set permet lui l'ajustement du modèle en agissant sur la modification des hyperparamètres

> Le test set permet de fournir une évaluation impartiale de notre modèle (ce jeu de données n'a pas servit à entraîner le modèle)

## Création d’un modèle de TA

In [ ]:
!onmt_build_vocab -config config-base.yaml -n_sample -1

**Q4**

> 

**Q5**

> Ces lignes nous donnent le nombre de mots que nos langages source et target contiennent. <span style="color:red">plus précisement: le nombre de tokens uniques présents dans le vocabulaire source et cible.</span> 

In [ ]:
%%shell
onmt_train -config config-baseQ6.yaml

**Q6**

> acc correspond précision du modèle en pourcentage (ratio succès sur tentatives). Il faut donc que cette valeur soit maximale.

> ppl correspond à la perplexité : La perplexité est une mesure de la capacité d'une distribution de probabilité ou d'un modèle de probabilité à prédire un échantillon. Plus elle est petite, mieux le modèle est performant.

**Q7**

- train_step : le nombre d'entraînements du réseaux <span style="color:red">Nombre total d'étapes d'optimisation des poids du modèle, où chaque étape est constitué d'un passage en avant d'un lot (batch) de données d'apprentissage dans le modèle (forward pass) et d'un passage en arrière des gradients générés par la fonction de perte (backward pass). </span>
- valid_step : le pas auquel on teste notre modèle sur les données de validations <span style="color:red">nombre d'étapes d'entrainement entre une étape de validation et la suivante. </span>
- enc_layers : nombre de couche de l'encodeur
- dec_layers : nombre de couche du décodeur
- enc_rnn_size : nombre de neurones par couche pour l'encodeur
- dec_rnn_size : nombre de neurones par couche pour le décodeur
- batch_size : nombre de phrases maximum processées à chaque itération (cf coefficient d'homogénéisation des données que l'on procède) <span style="color:red">itération de quoi? </span>



##  Evaluation d’un modèle de TA

In [ ]:
!onmt_translate -model models/baseQ6/model_step_625.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.tok.txt -output models/Q6_pred625.txt
!onmt_translate -model models/baseQ6/model_step_1250.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.tok.txt -output models/Q6_pred1250.txt
!onmt_translate -model models/baseQ6/model_step_1875.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.tok.txt -output models/Q6_pred1875.txt
!onmt_translate -model models/baseQ6/model_step_2000.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.tok.txt -output models/Q6_pred2000.txt

In [ ]:
!perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt_out.tok.txt < models/Q6_pred625.txt
!perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt_out.tok.txt < models/Q6_pred1250.txt
!perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt_out.tok.txt < models/Q6_pred1875.txt
!perl multi-bleu.perl ./BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt_out.tok.txt < models/Q6_pred2000.txt

BLEU = 8.57, 46.2/15.2/8.2/2.4 (BP=0.790, ratio=0.809, hyp_len=2892, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 13.98, 44.5/18.0/10.4/4.6 (BP=1.000, ratio=1.035, hyp_len=3698, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 17.96, 53.0/24.0/15.1/7.9 (BP=0.908, ratio=0.912, hyp

**Q8**
> Le score BLEU est compris entre 0 et 100%. On cherche à avoir ce score le plus proche possible de 100% car ce score indique dans quelle mesure le texte candidat est similaire aux textes de référence et donc en conséquence les valeurs plus proches de 100% représentant des textes plus similaires.

**Q9**
> Le model qui obtient le meilleur score est le dernier. Celui enregistrer à la fin de l'étape 2000.

## Optimisation des paramètres



**Q10**
> Nous testons dans un premier temps d'augmenter le nombre d'étapes à 5000 et procédons à l'entrainement.

In [ ]:
%%shell
onmt_train -config config-baseQ10.yaml

Evaluons le dernier modèle enregistré après les 5000 étapes =>

In [ ]:
!onmt_translate -model models/baseQ10/model_step_5000.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q10_pred5000.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q10_pred5000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 18:32:08,631 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 18:32:13,212 INFO] PRED AVG SCORE: -0.6617, PRED PPL: 1.9381
BLEU = 29.92, 64.3/37.6/25.7/16.6 (BP=0.938, ratio=0.940, hyp_len=3579, ref_len=3808)
It is 

> Nous sommes passé de 19.01% pour 2000 steps à 29.92% pour 5000 steps. Le BLEU augmente plus le nombre de training steps augmente. Il augmente donc en cohérence avec le principe d'apprentissage des réseaux de neurones. Plus on entraîne le modèle, plus il est performant.

> Cependant à partir d'un certain nombre d'entraînement, la performance du modèle n'augmente plus, les données d'entraînement ne suffisent plus pour permettre une amélioration. Il y a donc un nombre d'entraînement optimal à partir duquel le modèle ne s'améliore plus. Nous alons le trouver à partir de l'option `early7.2.3_stopping`. Nous allons donc mettre un très grand nombre de training steps et voir à quel moment le modèle arrète de s'entraîner suite à l'absence d'amélioration.


In [ ]:
%%shell
onmt_train -config config-baseQ10-1.yaml -early_stopping 2

> Dans notre cas, le modèle a arreté de s'entrainer après 2 étapes de validation consécutives sans amélioration.

> Calculons le BLEU correspondant au dernier modèle enregistré.

In [ ]:
!onmt_translate -model models/baseQ10_1/model_step_6875.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q10_1_pred6875.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q10_1_pred6875.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 18:32:56,484 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 18:33:00,946 INFO] PRED AVG SCORE: -0.5624, PRED PPL: 1.7548
BLEU = 34.47, 68.1/42.5/30.4/21.6 (BP=0.927, ratio=0.930, hyp_len=3541, ref_len=3808)
It is 

> Le score maximal que l'on arrive donc a obtenir en augmentant le nombre de training steps est 34.47%

**Q11**
> Relançons donc l'entrainement avec 3 couches sur l'encodeur.

In [ ]:
%%shell
onmt_train -config config-baseQ11.yaml

In [ ]:
!onmt_translate -model models/baseQ11/model_step_2000.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q11_pred2000.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q11_pred2000.txt

[2022-02-27 18:33:48,906 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 18:33:53,532 INFO] PRED AVG SCORE: -0.9749, PRED PPL: 2.6509
BLEU = 18.74, 55.8/26.4/15.9/8.1 (BP=0.899, ratio=0.903, hyp_len=3440, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores fro

> Nous obtenons un score BLEU de 18.74%. Augmenter les couches de l'encodeur de semble pas augmenter la performance du modèle.

> Testons à présent d'augmenter le nombre de couche du décodeur.

In [ ]:
%%shell
onmt_train -config config-baseQ11-1.yaml

In [ ]:
!onmt_translate -model models/baseQ11_1/model_step_2000.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q11_1_pred2000.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q11_1_pred2000.txt

[2022-02-27 18:34:15,998 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 18:34:22,652 INFO] PRED AVG SCORE: -1.2544, PRED PPL: 3.5056
BLEU = 11.41, 41.1/16.0/8.5/3.0 (BP=1.000, ratio=1.028, hyp_len=3915, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from

> Nous obtenons à présent un score de 11.41%. Essayons d'augmenter le nombre de steps pour voir si on peut avoir un meilleur score BLEU.

In [ ]:
%%shell
onmt_train -config config-baseQ11-2.yaml -early_stopping 2

In [ ]:
!onmt_translate -model models/baseQ11_2/model_step_10625.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q11_2_pred10625.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q11_2_pred10625.txt

[2022-02-27 18:34:48,745 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 18:34:54,689 INFO] PRED AVG SCORE: -0.6172, PRED PPL: 1.8537
BLEU = 28.81, 60.4/34.6/24.0/16.0 (BP=0.963, ratio=0.964, hyp_len=3671, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores fr

> On obtient un score BLEU de 28,81%. Augmenter le nombre de couche à 3 ne semble pas améliorer notre modèle.

> Testons avec 2 couches :

In [ ]:
%%shell
onmt_train -config config-baseQ11-3.yaml

In [ ]:
!onmt_translate -model models/baseQ11_3/model_step_2000.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q11_3_pred2000.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q11_3_pred2000.txt

[2022-02-27 19:08:41,801 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 19:08:47,178 INFO] PRED AVG SCORE: -1.0243, PRED PPL: 2.7851
BLEU = 14.63, 51.1/21.7/12.3/5.0 (BP=0.906, ratio=0.910, hyp_len=3467, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores fro

> Avoir 2 couches ne semble toujours pas améliorer le modèle.

**Q12**

Relançons donc l'entrainement avec différents nombres unités sur l'encodeur et le décodeur.

In [ ]:
%%shell
for i in 256 300 350 400 450 500
do
  onmt_train -config config-baseQ12_${i}.yaml
done


In [ ]:
%%shell
for i in 256 300 350 400 450 500
do
  onmt_translate -model models/baseQ12_${i}/model_step_2000.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q12_${i}_pred2000.txt
done
for i in 256 300 350 400 450 500
do
  perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q12_${i}_pred2000.txt
done

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 18:55:43,348 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 18:55:47,587 INFO] PRED AVG SCORE: -0.9397, PRED PPL: 2.5592
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

> On obtient comme maximum 20,48% pour 450 unités contre 18,83% pour 256 unités. Augmenter le nombre d'unités semble permettre une petite amélioration du modèle mais trop l'augmenter (ex : 500) ne semble pas apporter grand chose.

<span style="color:red">Vous auriez du donner des explications pour vos observations. Aussi, il aurait fallu utiliser l'option early_stopping pour bien repondre à cette question (ainsi que celle précédente). En effet, si l'on utilise pas cette option, on risque de comparer des modèles qui sont sous-entrainé ou sur-entrainés (overfitted),  sans donc pouvoir en tirer des conclusions. En effet, le nombre d'étapes d'entrainement nécessaires à l'optimization du modèle varie selon la quantité de paramétres (poids) à optimizer, ce qui est le cas lorsqu'on varie le nombre de couches ou leur taille.</span>

**Q14**
Le beam search est un algorithme de recherche heuristique qui explore un graphe en ne considérant qu'un ensemble limité de fils de chaque nœud. Cet algorithme est une optimisation de l'algorithme de parcours en largeur, en réduisant la mémoire nécessaire à son exécution.

<span style="color:red"> C'est quoi les noeuds et les fils dans notre cas ? (les tokens de chaque étape de décodage de la phrase cible, c.à.d. de la traduction) </span>

La valeur par défault est 5.

**Q15**
Faisont varier cette valeur :

In [ ]:
%%shell
for i in {1..10}
do
  onmt_translate --beam_size $i -model models/baseQ6/model_step_2000.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q14_${i}_pred2000.txt
done
for i in {1..10}
do
  perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q14_${i}_pred2000.txt
done


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 19:09:55,896 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 19:09:57,770 INFO] PRED AVG SCORE: -1.1265, PRED PPL: 3.0848
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

> On remarque que le score BLEU croît jusqu'à la valeur du beam à 10.

<span style="color:red"> Pourquoi? </span>

**Q16**
> Testons d'obtenir le meilleur score BLEU avec les paramètres suivants :

* Un maximum de training steps avec l'option d'arrêt automatique
* 1 couche pour l'encodeur et 1 couche pour le décodeur
* 450 d'unités pour l'encodeur et le décodeur
* Un beam search de taille 10




In [ ]:
%%shell
onmt_train -config config-baseQ16.yaml -early_stopping 2

In [ ]:
!onmt_translate --beam_size 10 -model models/baseQ16/model_step_6875.pt -src ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt_out.tok.txt -output models/Q16_pred6875.txt
!perl multi-bleu.perl ./BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt_out.tok.txt < models/Q16_pred6875.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-27 19:37:05,510 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-27 19:37:19,735 INFO] PRED AVG SCORE: -0.4875, PRED PPL: 1.6283
BLEU = 35.67, 67.9/42.7/30.8/22.1 (BP=0.952, ratio=0.953, hyp_len=3628, ref_len=3808)
It is 

> On obtient le score de 35.67%


**Q17**

Regardons les prédiction sur le test set pour le comparer au model de la Q6.


In [ ]:
!onmt_translate --beam_size 10 -model models/baseQ16/model_step_6875.pt -src ./BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt_out.tok.txt -output models/Q16_pred6875_test.txt


**Exemples**

Target : How long will we have to wait ? (line 11)

Q6 : How long will it take ?

Q17 : How long will we have to wait ?

Target : Every ten minutes . (line 17)

Q6 : All three days .

Q17 : Every ten minutes .

Target : Could I get you some coffee or tea ? (line 78)

Q6 : Could I have a blanket and a blanket ?

Q17 : Could I have a coffee or a coffee ?

Target : I believe that this is my seat .
(line 233)

Q6 : I think this is my room .

Q17 : I think this is my seat .